In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
data_ll = pd.read_excel("../Paris housing Data Set 2 4050.xlsx")
data = data_ll.copy(deep=True)
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_total,floors,condition,grade,built,renovated,living_area_sqft
0,221900.0,3.0,1.0,1180.0,5650.0,1.0,3.0,7,1955.0,0,1340.0
1,538000.0,3.0,NaN,2570.0,7242.0,2.0,3.0,7,1951.0,1991,1690.0
2,180000.0,2.0,1.0,770.0,10000.0,1.0,3.0,6,1933.0,0,2720.0
3,604000.0,4.0,3.0,1960.0,5000.0,1.0,5.0,7,1965.0,0,1360.0
4,510000.0,3.0,2.0,1680.0,8080.0,1.0,3.0,8,1987.0,0,1800.0


In [24]:
#data.isna().sum()

In [25]:
data_without_price = data[data.price.isna()]
data_without_price.reset_index(inplace=True)
dict_from_df_index = dict(zip( data_without_price.index, data_without_price["index"]))
data_without_price.drop(columns=['index'], inplace=True)
# print(dict_from_df_index)
# data_without_price

C:\Users\HP\AppData\Local\Temp\ipykernel_15392\4027092775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_without_price.drop(columns=['index'], inplace=True)


In [26]:
data.price.describe()

count    1.999500e+04
mean     5.353944e+05
std      3.659217e+05
min      7.500000e+04
25%      3.170000e+05
50%      4.499000e+05
75%      6.400000e+05
max      7.700000e+06
Name: price, dtype: float64

In [27]:
# drop all rows with nan from data
data_without_nan = data.dropna()
data_without_nan.isna().sum()

price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_total          0
floors              0
condition           0
grade               0
built               0
renovated           0
living_area_sqft    0
dtype: int64

### ONLY CONSIDER COLUMNS WITH A HIGH CORELLATION TO PRICE (>= 0.5)

In [28]:
column_list = ['price','grade','bathrooms','sqft_living','living_area_sqft']
data_without_price = data_without_price[column_list]
data_without_price

,price,grade,bathrooms,sqft_living,living_area_sqft
0,NaN,9,2.25,2240.0,2440.0
1,NaN,7,1.50,1140.0,1200.0
2,NaN,10,2.75,3330.0,2390.0
3,NaN,10,2.50,2880.0,2640.0


In [29]:
row_data = data_without_price.iloc[0].values.tolist()
row_data[1:]

[9.0, 2.25, 2240.0, 2440.0]

In [30]:
# filter out only our target columns
data_without_nan = data_without_nan[column_list]
filtered_df = data_without_nan[
    (data_without_nan["grade"].between(7, 10)) & 
    (data_without_nan["bathrooms"].between(1, 3)) & 
    (data_without_nan["sqft_living"].between(1100, 3340)) & 
    (data_without_nan["living_area_sqft"].between(1190, 26650))
].reset_index(drop=True)  # Reset index
 
filtered_df

,price,grade,bathrooms,sqft_living,living_area_sqft
0,221900.0,7,1.00,1180.0,1340.0
1,604000.0,7,3.00,1960.0,1360.0
2,510000.0,8,2.00,1680.0,1800.0
3,257500.0,7,2.25,1715.0,2238.0
4,229500.0,7,1.00,1780.0,1780.0
...,...,...,...,...,...
14014,439108.0,8,1.50,1130.0,1310.0
14015,260000.0,7,2.25,1489.0,1584.0
14016,279000.0,7,2.50,2690.0,2090.0
14017,380000.0,7,2.25,1530.0,1320.0


In [31]:
# # Target row
# target_row = np.array([row_data[1:]])  # Convert to 2D array

# # Compute cosine similarity
# similarities = cosine_similarity(filtered_df.iloc[:,1:].values, target_row)

# # Add similarity scores to the DataFrame
# filtered_df['similarity'] = similarities.round(2)

# # Sort by highest similarity
# df_sorted = filtered_df.sort_values(by='similarity', ascending=False)
# #df_sorted = df_sorted[df_sorted['similarity'] > 0.9999]
# df_sorted

In [32]:
#[nan, 9.0, 2.25, 2240.0, 2440.0]
# new_df = df_sorted[
#     (df_sorted["grade"] == 9.0) & 
#     (df_sorted["bathrooms"].between(2.0,2.5)) & 
#     (df_sorted["sqft_living"].between(2200, 2300)) & 
#     (df_sorted["living_area_sqft"].between(2400, 2500))
# ]
# new_df

In [33]:
# most correlated value with price is sqft_living and we want to get the house closest to our target in terms of sqft_living
# df_sorted["rank"] = (df_sorted["sqft_living"] - 2240).abs().rank(method="min")
# new_df = df_sorted.sort_values(by="rank")
# new_df



In [34]:
# get row index from df
# get the value of sqft_living
# run the cosine_similarity and sort df with similarit
# cut off from similarity >= 0.99
# rank them by how far they are from the target sqft_living of the row

In [35]:
row_data = data_without_price.iloc[0].tolist()
row_data
index_list = data_without_price.index.tolist()
for val in index_list:
    orig_data_index = dict_from_df_index[val]
    target_df = filtered_df
    sqft_living_value = data_without_price['sqft_living'].iloc[val].item()

    # Target row
    row_data = data_without_price.iloc[val].tolist()
    target_row = np.array([row_data[1:]])  # Skip the price

    # Compute cosine similarity
    if 'similarity' in target_df.columns:
        target_df = target_df.drop(columns=['similarity'])
    similarities = cosine_similarity(target_df.iloc[:,1:].values, target_row) # also exclude price here

    # Add similarity scores to the DataFrame
    target_df['similarity'] = similarities.round(2)

    # Sort by highest similarity
    df_sorted = target_df.sort_values(by='similarity', ascending=False)

    # most correlated value with price is sqft_living and we want to get the house closest to our target in terms of sqft_living
    df_sorted["rank"] = (df_sorted["sqft_living"] - sqft_living_value).abs().rank(method="min")
    new_df = df_sorted.sort_values(by="rank")
    #print(new_df.head(2))

    # set price value on the data_without_price_df. always pick the row with rank =1, index =0
    # set price value on the original df(data). always pick the row with rank =1, index =0
    data_without_price.at[val, "price"] = new_df['price'].iloc[0].item()
    data.at[orig_data_index, "price"] = new_df['price'].iloc[0].item()

    #del df_sorted, target_df
print(data.isna().sum())   
data_without_price



price                0
bedrooms            15
bathrooms           11
sqft_living          1
sqft_total           5
floors               0
condition            1
grade                0
built                1
renovated            0
living_area_sqft     6
dtype: int64


,price,grade,bathrooms,sqft_living,living_area_sqft
0,410000.0,9,2.25,2240.0,2440.0
1,345000.0,7,1.50,1140.0,1200.0
2,1824100.0,10,2.75,3330.0,2390.0
3,1989000.0,10,2.50,2880.0,2640.0


In [36]:
data.price.describe().T.round(2)

count      19999.00
mean      535515.74
std       366146.37
min        75000.00
25%       317000.00
50%       449900.00
75%       640000.00
max      7700000.00
Name: price, dtype: float64

In [37]:
data.price.value_counts

<bound method IndexOpsMixin.value_counts of 0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
19994    260000.0
19995    279000.0
19996    380000.0
19997    287000.0
19998    377000.0
Name: price, Length: 19999, dtype: float64>

In [38]:
len(data[data['price']>1200000])

885

In [39]:
len(data[(data["price"].between(1000000,1200000))])

462

In [40]:
len(data[(data["price"].between(800000,1000000))])

1406